## IMPORTING PACKAGES 

In [ ]:
import pandas as pd
import re
import warnings
import enchant
import emojis
import langid
from collections import Counter
from cleantext import clean
from googletrans import Translator
warnings.filterwarnings('ignore')

## DATA PREPROCESSING

In [ ]:
def basic(sp1):
    
    sp1['date'] = sp1['date'].str.replace('Reviewed in India on ', '')
    sp1['rating'] = sp1['rating'].str.replace(' out of 5 stars', '')
    sp1['title'] = sp1['title'].str.replace('\n', '')
    sp1['review'] = sp1['review'].str.replace('\n', '')
    sp1['review']=sp1['review'].str.lower()
    sp1['title']=sp1['title'].str.lower()
    return sp1

def null_values(sp1):  
    index=sp1.loc[sp1["review"].isna()]["review"].index
    for i in index:
        sp1["review"][i]=sp1["title"][i]
    return sp1

def remove_emoji(sp1):
    for i in range(len(sp1['review'])):
        sp1['review'][i]=clean(sp1['review'][i], no_emoji=True)
    return sp1

def translation(sp1):
    o=[]
    translator = Translator()
    for i in range(len(sp1['review'])):
        a=langid.classify(sp1['review'][i])
        if a[0]=='ta' or a[0]=='hi':
            o.append(i)
    for j in o:
        sp1["review"][j]=translator.translate(sp1['review'][j]).text  
    return sp1

    
def tokenize(sp1):   
    for i in range(len(sp1['review'])):
        sp1['review'][i]=re.findall(r"(?<![@#])\b\w+(?:'\w+)?",sp1['review'][i])
    return sp1

dict = enchant.Dict("en_US")
def possible_words(word):
    words=[word]
    for i in range(len(word)-1):
        w1=word[:i]+word[i+1]+word[i]+word[i+2:]
        w2=word[:i]+word[i+1:]
        w3=word[:i+1]+word[i+2:]
        if w1 not in words:
            words.append(w1)
        if w2 not in words:
            words.append(w2)
        if i==len(word)-2 and w3 not in words:
            words.append(w3)
    letters='abcdefghijklmnopqrstuvwxyz'
    w=[]
    for i in words:
        if dict.check(i):
            w.append(i)
    for k in words:
        for i in range(len(k)):
            for j in range(len(letters)):
                k1=k[:i]+letters[j]+k[i:]
                k2=k[:i]+letters[j]+k[i+1:]
                if k[:i]+letters[j]+k[i:] not in w and dict.check(k1):
                    w.append(k1)
                    if i<len(k) and k[:i]+letters[j]+k[i+1:] not in w and dict.check(k2):
                        w.append(k2)
    return w

def sf_check(word):
    df=pd.read_csv("wtsapp.txt",delimiter="\t")
    for i in range(len(df)):
        if word==df["sf"][i]:
            return df["ff"][i]
    return False

def split_words(a):
    count=1
    l=a
    for i in range(len(a)+1):
        a1=sf_check(a[i:])
        a2=sf_check(a[:i])
        if len(a[:i])>0 and len(a[i:])>0 and dict.check(a[:i]) and dict.check(a[i:]):      
            l=a[:i]+" "+a[i:]
            break
        elif len(a[:i])>0 and dict.check(a[:i]):
            if a1:
                if len(a[:i])==0:                                
                    l=a1
                l=a[:i]+" "+a1
                break
            l=a[:i]+" "+a[i:]
        elif len(a[i:])>0 and dict.check(a[i:]):
            if a2:
                if len(a[i:])==0:
                    l=a2
                l=a2+" "+a[i:]
                break
            l=a[:i]+" "+a[i:]
        elif a2 and a1:
            l=a2+" "+a1    
            break
    return l

def P(word): 
    r=open(r"big.txt").read()
    r=Counter(re.findall(r'\w+', r.lower()))
    return r[word]
                        
def spellcheck(col):
    l=[]
    for st in range(len(col)):
        l=l+col[st]
    l=Counter(l)
    r=open(r"big.txt").read()
    r=Counter(re.findall(r'\w+', r.lower()))
    for i in range(len(col)):
        for j in range(len(col[i])):
            word=col[i][j]
            if dict.check(word)==False:
                w_check=sf_check(word)
                if w_check:
                    col[i][j]=w_check
                    continue
                count=0
                count1=0
                c={}
                s=0
                a=possible_words(word)
                list1={}
                for k in a:
                    count_1=0
                    if k in l.keys():
                        list1[k]=l[k]
                        col[i][j]=max(list1, key=list1.get)
                        count1=1 
                        s=1
                        break
                    elif k in r.keys():
                        c[k]=P(k)  
                        count+=1
                if count>0 and s==0:
                    count1=1 
                    col[i][j]=max(c, key=c.get)
                if not count1==1:
                    col[i][j]=split_words(word)
                
    return col                 
    

def negation_handling(sp1):
    
    neg=["ain'",'aint',"aren'", "arent", 'angry','annoy','anxious','atrocious','awful','bad','boring','broken','cant','cannot','clumsy','coarse',
     'collapse','confused','corrosive','couldn','couldnt','corrupt','crazy','creepy','criminal','cruel','cry','cutting','damage',
     'dead','decaying','deny','depressed','didn', 'didnt', "doesn'", 'doesnt','dirty','disgusting','dishonest','dishonorable',
     'don','dont','dreadful','eroding','evil','fail','faulty','filthy','few','frighten','gross','hate','hideous','hard','harmful',
     'horrible','hurt','hadn', 'hadnt', 'hasn', 'hasnt', 'haven', 'havent','ignore','immature','imperfect','impossible',
     'isn','isnt','inelegant','malicious','mean','messy','misshapen','missing','misunderstood','mightn', 'mightnt', 'mustn',
     'mustnt', 'needn','neednt','nasty','never','no','nor','nobody','nonsense','not','none','noone','nothing','neither',
     'nowhere','odious','offensive','old','poor','reject','sad','scary','shocking','severe','sick','stressful','stuck','stupid',
     'suspect','suspicious','shan', 'shant', 'shouldn', 'shouldnt','terrible','terrifying','threatening','ugly','unfair',
     'unhappy','unlucky','unpleasant','unsatisfactory','upset','worthless','wasn', 'wasnt', 'weren', 'werent', 'won', 'wont',
     'wouldn', 'wouldnt','yucky','zero','disappointed','embarrassed','insane','loss','nothing','missing','hardly','scarcely',
     'barely']

    stop_words=['a', 'about', 'above', 'across', 'after', 'again', 'against', 'all', 'alone', 'along', 'already', 
            'although', 'always', 'among', 'an', 'and', 'another', 'any', 'anybody', 'anyone', 'anything', 
            'anywhere', 'are', 'area', 'areas', 'around', 'as', 'ask', 'asked', 'asking', 'asks', 'at', 'away',
            'b','but', 'back', 'backed', 'backing', 'backs', 'be', 'became', 'because', 'become', 'becomes', 'been', 
            'before', 'began', 'behind', 'being', 'beings', 'between', 'big', 'both', 'by', 'c', 'came', 'can', 
            'cannot', 'case', 'cases', 'certain', 'certainly', 'clear', 'clearly', 'come', 'could', 'd', 'did', 
            'differ', 'different', 'differently', 'do', 'does', 'done', 'down', 'downed', 'downing', 'downs', 
            'during', 'e', 'each', 'early', 'either', 'end', 'ended', 'ending', 'ends', 'enough', 'even', 'evenly', 
            'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'f', 'face', 'faces', 'fact', 'facts',
            'far', 'felt', 'few', 'find', 'finds', 'first', 'for', 'four', 'from', 'full', 'fully', 'further', 'furthered',
            'furthering', 'furthers', 'g', 'gave', 'general', 'generally', 'get', 'gets', 'give', 'given', 'gives', 
            'go', 'going', 'got', 'group', 'grouped', 'grouping', 'groups', 'h', 'had', 'has', 'have', 'having', 'he', 
            'her', 'here', 'herself', 'him', 'himself', 'his', 'how', 'however', 'i', 'if', 'in', 'into', 'is', 'it', 
            'its', 'itself', 'j', 'just', 'k', 'keep', 'keeps', 'kind', 'knew', 'know', 'known', 'knows', 'l', 'large', 
            'largely', 'last', 'later', 'let', 'lets', 'long', 'longer', 'longest', 'm', 'made', 'make', 'making', 'man',
            'many', 'may', 'me', 'member', 'members', 'men', 'might', 'mr', 'mrs', 'much', 'must', 'my', 'myself', 
            'n', 'necessary', 'need', 'needed', 'needing', 'needs', 'never', 'next', 'no', 'nobody', 'none', 'noone', 
            'not', 'nothing', 'now', 'nowhere', 'number', 'numbers', 'o', 'of', 'off', 'often', 'on', 'once', 'one', 
            'open', 'opened', 'opening', 'opens', 'or', 'other', 'others', 'our', 'out', 'over', 'p', 'part', 'parted', 
            'parting', 'parts', 'per', 'perhaps', 'place', 'places', 'point', 'pointed', 'pointing', 'points', 'possible', 
            'present', 'presented', 'presenting', 'presents', 'problem', 'problems', 'put', 'puts', 'q', 'quite', 'r', 
            'rather', 'really', 'room', 'rooms', 's', 'said', 'same', 'saw', 'say', 'says', 'second', 'seconds', 'see', 
            'seem', 'seemed', 'seeming', 'seems', 'sees', 'several', 'shall', 'she', 'should', 'show', 'showed', 'showing',
            'shows', 'side', 'sides', 'since', 'so', 'some', 'somebody', 'someone', 'something', 'somewhere', 'state', 
            'states', 'still', 'such', 'sure', 't', 'take', 'taken', 'than', 'that', 'the', 'their', 'them', 'then', 'there',
            'therefore', 'these', 'they', 'thing', 'things', 'think', 'thinks', 'this', 'those', 'though', 'thought',
            'thoughts', 'three', 'through', 'thus', 'to', 'today', 'together', 'took', 'toward', 'turn', 'turned', 'turning', 
            'turns', 'two', 'u', 'under', 'until', 'up', 'upon', 'us', 'use', 'used', 'uses', 'v', 'w', 'want', 'wanted',
            'wanting', 'wants', 'was', 'way', 'ways', 'we', 'wells', 'went', 'were', 'what', 'when', 'where', 'whether', 
            'which', 'while', 'who', 'whole', 'whose', 'why', 'will', 'with', 'within', 'work', 'would', 'x', 'y', 'year',
            'years', 'yet', 'you', 'young', 'younger', 'youngest', 'your', 'yours', 'z', 'whereas', 'afterwards', 'wherein', 
            'eg', 'twelve', 'yourselves', 'hereupon', 'mostly', 'eleven', 'therein', 'only', 'else', 'yourself', 'sixty', 
            'nine', 'latterly', 'whatever', 'describe', 'fill', 'hereafter', 'six', 'name', 'whence', 'becoming', 'well',
            'due', 'etc', 'thick', 'whereafter', 'interest', 'whereupon', 'thereupon', 'themselves', 'thereby', 'un', 'five',
            'sometimes', 'via', 'former', 'hers', 'whenever', 'hundred', 'besides', 'detail', 'serious', 'almost', 'throughout', 
            'sometime', 'beyond', 'anyway', 'towards', 'moreover', 'fire', 'least', 'beside', 'co', 'thin', 'found', 'without',
            'inc', 'am', 'except', 'sincere', 'whoever', 'most', 'twenty', 'thru', 'below', 'wherever', 'own', 'eight', 'ltd', 
            'herein', 'top', 'con', 'mine', 'third', 'thereafter', 'too', 'also', 'indeed', 'whom', 'fifteen', 'hence', 'ourselves', 
            'elsewhere', 'amongst', 'beforehand', 'namely', 'more', 'whither', 'somehow', 'empty', 'ie', 'front', 'bill', 'please', 
            'very', 'anyhow', 'cry', 'amoungst', 'ours', 'meanwhile', 'formerly', 'fifty', 'bottom', 'otherwise', 'move', 'thence',
            'mill', 'forty', 'less', 'ten', 'latter', 'whereby']
    print(len(stop_words))
    for i in neg:
        if i in stop_words:
            stop_words.remove(i)
    print(len(stop_words))
    return sp1

    
    
def stemming(sp1):
    for i in range(len(sp1['review'])):
        s=[]
        for word in sp1['review'][i]:
            s.append(re.sub(r'less|ship|ships|ment|ing|les|ly|ed|ion|ous|able|', '', word))
        sp1['review'][i]=s
    return sp1
    

def stw(col):   
    sw=['a', 'about', 'above', 'across', 'after', 'again', 'against', 'all', 'alone', 'along', 'already', 
            'although', 'always', 'among', 'an', 'and', 'another', 'any', 'anybody', 'anyone', 'anything', 
            'anywhere', 'are', 'area', 'areas', 'around', 'as', 'ask', 'asked', 'asking', 'asks', 'at', 'away',
            'b','but', 'back', 'backed', 'backing', 'backs', 'be', 'became', 'because', 'become', 'becomes', 'been', 
            'before', 'began', 'behind', 'being', 'beings', 'between', 'big', 'both', 'by', 'c', 'came', 'can', 
            'cannot', 'case', 'cases', 'certain', 'certainly', 'clear', 'clearly', 'come', 'could', 'd', 'did', 
            'differ', 'different', 'differently', 'do', 'does', 'done', 'down', 'downed', 'downing', 'downs', 
            'during', 'e', 'each', 'early', 'either', 'end', 'ended', 'ending', 'ends', 'enough', 'even', 'evenly', 
            'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'f', 'face', 'faces', 'fact', 'facts',
            'far', 'felt', 'few', 'find', 'finds', 'first', 'for', 'four', 'from', 'full', 'fully', 'further', 'furthered',
            'furthering', 'furthers', 'g', 'gave', 'general', 'generally', 'get', 'gets', 'give', 'given', 'gives', 
            'go', 'going', 'got', 'group', 'grouped', 'grouping', 'groups', 'h', 'had', 'has', 'have', 'having', 'he', 
            'her', 'here', 'herself', 'him', 'himself', 'his', 'how', 'however', 'i', 'if', 'in', 'into', 'is', 'it', 
            'its', 'itself', 'j', 'just', 'k', 'keep', 'keeps', 'kind', 'knew', 'know', 'known', 'knows', 'l', 'large', 
            'largely', 'last', 'later', 'let', 'lets', 'long', 'longer', 'longest', 'm', 'made', 'make', 'making', 'man',
            'many', 'may', 'me', 'member', 'members', 'men', 'might', 'mr', 'mrs', 'much', 'must', 'my', 'myself', 
            'n', 'necessary', 'need', 'needed', 'needing', 'needs', 'never', 'next', 'no', 'nobody', 'none', 'noone', 
            'not', 'nothing', 'now', 'nowhere', 'number', 'numbers', 'o', 'of', 'off', 'often', 'on', 'once', 'one', 
            'open', 'opened', 'opening', 'opens', 'or', 'other', 'others', 'our', 'out', 'over', 'p', 'part', 'parted', 
            'parting', 'parts', 'per', 'perhaps', 'place', 'places', 'point', 'pointed', 'pointing', 'points', 'possible', 
            'present', 'presented', 'presenting', 'presents', 'problem', 'problems', 'put', 'puts', 'q', 'quite', 'r', 
            'rather', 'really', 'room', 'rooms', 's', 'said', 'same', 'saw', 'say', 'says', 'second', 'seconds', 'see', 
            'seem', 'seemed', 'seeming', 'seems', 'sees', 'several', 'shall', 'she', 'should', 'show', 'showed', 'showing',
            'shows', 'side', 'sides', 'since', 'so', 'some', 'somebody', 'someone', 'something', 'somewhere', 'state', 
            'states', 'still', 'such', 'sure', 't', 'take', 'taken', 'than', 'that', 'the', 'their', 'them', 'then', 'there',
            'therefore', 'these', 'they', 'thing', 'things', 'think', 'thinks', 'this', 'those', 'though', 'thought',
            'thoughts', 'three', 'through', 'thus', 'to', 'today', 'together', 'took', 'toward', 'turn', 'turned', 'turning', 
            'turns', 'two', 'u', 'under', 'until', 'up', 'upon', 'us', 'use', 'used', 'uses', 'v', 'w', 'want', 'wanted',
            'wanting', 'wants', 'was', 'way', 'ways', 'we', 'wells', 'went', 'were', 'what', 'when', 'where', 'whether', 
            'which', 'while', 'who', 'whole', 'whose', 'why', 'will', 'with', 'within', 'work', 'would', 'x', 'y', 'year',
            'years', 'yet', 'you', 'young', 'younger', 'youngest', 'your', 'yours', 'z', 'whereas', 'afterwards', 'wherein', 
            'eg', 'twelve', 'yourselves', 'hereupon', 'mostly', 'eleven', 'therein', 'only', 'else', 'yourself', 'sixty', 
            'nine', 'latterly', 'whatever', 'describe', 'fill', 'hereafter', 'six', 'name', 'whence', 'becoming', 'well',
            'due', 'etc', 'thick', 'whereafter', 'interest', 'whereupon', 'thereupon', 'themselves', 'thereby', 'un', 'five',
            'sometimes', 'via', 'former', 'hers', 'whenever', 'hundred', 'besides', 'detail', 'serious', 'almost', 'throughout', 
            'sometime', 'beyond', 'anyway', 'towards', 'moreover', 'fire', 'least', 'beside', 'co', 'thin', 'found', 'without',
            'inc', 'am', 'except', 'sincere', 'whoever', 'most', 'twenty', 'thru', 'below', 'wherever', 'own', 'eight', 'ltd', 
            'herein', 'top', 'con', 'mine', 'third', 'thereafter', 'too', 'also', 'indeed', 'whom', 'fifteen', 'hence', 'ourselves', 
            'elsewhere', 'amongst', 'beforehand', 'namely', 'more', 'whither', 'somehow', 'empty', 'ie', 'front', 'bill', 'please', 
            'very', 'anyhow', 'cry', 'amoungst', 'ours', 'meanwhile', 'formerly', 'fifty', 'bottom', 'otherwise', 'move', 'thence',
            'mill', 'forty', 'less', 'ten', 'latter', 'whereby']
    
    for i in range(len(col)):
        b=[]
        for j in col[i]:
            if j in sw:
                b.append(j)
        for j in b:
            col[i].remove(j)
    return col
